<a href="https://colab.research.google.com/github/silverstar0727/DSC-ML-Research-Study/blob/main/TF_to_torch_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
""" configuration json을 읽어들이는 class """
class Config(dict): 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

    @classmethod
    def load(cls, file):
        with open(file, 'r') as f:
            config = json.loads(f.read())
            return Config(config)

In [32]:
config = Config({
    'batch_size': 128,
    'learning_rate': 0.0002,
    'epochs': 500,
    'in_features': 256,
    'in_channels': 64
})

# Libraries

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

# Modeling
- generator

In [34]:
class Generator(nn.Module):
  def __init__(self, config):
    super(Generator, self).__init__()
    self.config = config

    # def layers
    self.fc1 = nn.Linear(in_features=self.config.in_features, out_features = 7*7*128)
    self.bn1 = nn.BatchNorm2d(7*7*128)
    self.conv2tr1 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=5, stride=2)
    self.bn2 = nn.BatchNorm2d(64)
    self.conv2tr2 = nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=5, stride=2)

  # def logic
  def forward(self, x):
    x = self.fc1(x)
    x = self.bn1(x)
    x = nn.LeakyReLU(x)
    x = x.view(-1, 7, 7, 128)
    x = self.conv2tr1(x)
    x = self.bn2(x)
    x = nn.LeakyReLU(x)
    x = self.conv2tr2(x)
    x = nn.tanh(x)

    return x

- discriminator

In [39]:
class Discriminator(nn.Module):
  def __init__(self, config):
    super(Discriminator, self).__init__()
    self.config = config

    self.conv1 = nn.Conv2d(in_channels = self.config.in_channels ,out_channels=64, kernel_size=5, stride = 2)
    self.bn1 = nn.BatchNorm2d(64)
    self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=5, stride = 2)
    self.bn2 = nn.BatchNorm2d(128)
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(in_features=2048, out_features=1024)
    self.bn3 = nn.BatchNorm2d(1024)
    self.fc2 = nn.Linear(in_features=1024, out_features = 2)

  def forward(self, x):
    x = x.view(-1, 28, 28, 1) 
    x = self.conv1(x)
    x = self.bn1(x)
    x = nn.LeakyReLU(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.bn3(x)
    x = nn.LeakyReLU(x)
    x = self.fc2(x)

    return x

# compile
(loss: cross entropy, optimizer: Adam)

In [43]:
generator = Generator(config)
discriminator = Discriminator(config)

# default
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 모델 올리기
generator = generator.to(device)
discriminator = discriminator.to(device)

# 손실함수 정하기
gen_loss_func = nn.CrossEntropyLoss()
dis_loss_func = nn.CrossEntropyLoss()

# 최적화 함수 정하기
gen_optimizer = torch.optim.Adam(generator.parameters(), lr = config.learning_rate) # 최적화 대상으로는 model.parameter를 정하기
dis_optimizer = torch.optim.Adam(discriminator.parameters(), lr = config.learning_rate)